# The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME")

client = openai.AzureOpenAI(
  api_key=azure_openai_api_key,
  api_version="2024-02-01",
  azure_endpoint=azure_openai_api_endpoint
)


def get_completion(prompt, model=deployment_name):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [2]:

def get_completion_from_messages(messages, model=deployment_name, temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message.content

In [3]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [4]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to yonder faire village, forsooth!


In [5]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! How may I assist you today?


In [6]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have access to that information as I don't have any memory storage. Could you please tell me your name?


In [7]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Sure, your name is Isa!


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [8]:
# initial system prompt

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


def collect_messages(prompt):
    
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
 
    return response


### Code pour avoir un serveur avec chatbot interactif 

In [12]:
import panel as pn  # GUI
import time

pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

    
def get_response(prompt, user, instance):
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    
    for index in range(len(response)):
        yield response[0:index+1]
        time.sleep(0.01)

chat_bot = pn.chat.ChatInterface(callback=get_response, max_height=500)
chat_bot.send("Hello! Welcome to our pizza restaurant. What can I get for you today?",
              user="Assistant",
              respond=False)
chat_bot.servable()

pn.serve(chat_bot, show=True)

Launching server at http://localhost:58199


### Code pour avoir un chatbot interactif dans le notebook :

Note: il est possible que lorsque vous tapiez du texte cela active certains raccourcis jupyter qui marchent avec une seule lettre, il faudra les désactiver.

In [10]:
import panel as pn

pn.extension()


def send_prompt(event=None):
    prompt = user_input.value
    response = collect_messages(prompt)
    response_area.value = f"> {prompt}\n{response}"

    conversation_history.value += f"USER: {user_input.value}\nChatGPT: {response}\n{'-' * 10}"


    user_input.value = ""

# Panel UI components
title = pn.pane.Markdown("# ChatGPT Panel Application")
title = pn.pane.Markdown("### Conversation History")
user_input = pn.widgets.TextInput(name="Type message here...")
send_button = pn.widgets.Button(name="Send", button_type="primary")

response_area = pn.widgets.TextAreaInput(value="", disabled=True, height=300, css_classes=["text-area-scrollbar"])
conversation_history = pn.widgets.TextAreaInput(value="", disabled=True, height=500, css_classes=["text-area-scrollbar"])

send_button.on_click(send_prompt)

# Arrange components and create the panel application
layout = pn.Column(
    title,
    pn.Row(response_area, conversation_history),
    pn.Row(user_input, send_button, width=600),
)

layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'fa5caa6a-6803-48cc-97b5-eee2b154aa64': {'version…

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

Sure, here's a summary of your order:

```
{
  "pizza": {
    "type": "pepperoni",
    "size": "large",
    "price": 12.95
  },
  "toppings": [],
  "drinks": [],
  "sides": [
    {
      "type": "fries",
      "size": "large",
      "price": 4.50
    }
  ],
  "total_price": 17.45
}
```

Please confirm if this is correct or if you'd like to add anything else to your order.


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

In [11]:
import panel as pn  # GUI
import time

pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

    
def get_response(prompt, user, instance):
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    
    for index in range(len(response)):
        yield response[0:index+1]
        time.sleep(0.01)

chat_bot = pn.chat.ChatInterface(callback=get_response, max_height=500)
chat_bot.send("Hello! Welcome to our pizza restaurant. What can I get for you today?",
              user="Assistant",
              respond=False)
chat_bot.servable()

pn.serve(chat_bot, show=True)

Launching server at http://localhost:58128
